In [1]:
import pickle
import pandas as pd
pd.set_option('display.max_columns', None)
# useful functions

# prints all the columns
def printcolumns(data):
    for column in data.columns:
        print (column)
        
# filters out data if a point is missing in one of the colunns
def filterblanks(columns,data,blank):
    # if blank is true, remove blanks
    # if blank is false, remove non blanks
    for c in columns:
        if blank:
            data = data[data[c].notnull()]
        else:
            data = data[data[c].isnull()]
    return data

In [6]:
#load data
data = pickle.load(open("water_data.p", "rb"))
print(data.shape)

(106052, 131)


In [7]:
# filter
print("Before filtering: ",data.shape)
#printcolumns(data)
data.drop(data.columns.difference(['TP','TPQF','TN','TNQF','SS','SSQF',
                                         'TURB','TURBQF','WDP',
                                         'TEMP','TEMPQF','DO','DOQF','COND',
                                         'CONDQF','VEL','VELQF','FLDEAST',
                                         'FLDNORTH','PROJCD','FLDNUM','DATE',
                                  'LOCATCD','STRATUM','CHLcal','SECCHI','SECCHIQF']), 1, inplace=True)
print("After filtering columns: ",data.shape)
print("Now filtering sampling design")
data = data[(data.PROJCD == "M-")]
print("After filtering sampling design: ",data.shape)
print("Now filtering Pool 13")
data = data[(data.FLDNUM == 3)]
print("After filtering Pool 13: ",data.shape)
print("Now adding a year column")
data["YEAR"] = pd.DatetimeIndex(data["DATE"]).year
print(data.shape)
print("Adding a timecode column")
data["TIME CODE"] = data["LOCATCD"].astype(str).apply(lambda x: x[3])
print(data.shape)
#print("Now filtering summer by timecode 2 (June Through August)")
#data = data[data["TIME CODE"] == '2']
#print(data.shape)
print("Now filtering by backwater lakes")
data = data[data.STRATUM == 3]
print(data.shape,data["YEAR"].unique())
print("Filtering out bad data")
QFcols = ['TPQF','TNQF','SSQF','TURBQF','TEMPQF','DOQF','CONDQF','VELQF','SECCHIQF']
data = filterblanks(QFcols,data,False)
print(data.shape,data["YEAR"].unique())
print("Dropping all blank columns")
data.drop(['PROJCD','FLDEAST','FLDNORTH','TPQF','TNQF','SSQF','TURBQF','TEMPQF','DOQF',
                                         'CONDQF','VELQF','SECCHIQF'], 1, inplace=True)
print(data.shape)
print("Filtering out points with blank entries in at least one of the columns")
cols = ['TP','TN','TEMP','DO','TURB','COND','VEL','SS','WDP','CHLcal','SECCHI']
data = filterblanks(cols,data,True)
print(data.shape,data["YEAR"].unique())
#print("Filtering out all years but 1993, 1994, 1995, 1996")
#data = data[(data['YEAR']==1993) | (data['YEAR']==1994) | (data['YEAR']==1995) | (data['YEAR']==1996)]
#print(data.shape)

Before filtering:  (106052, 131)
After filtering columns:  (106052, 27)
Now filtering sampling design
After filtering sampling design:  (106052, 27)
Now filtering Pool 13
After filtering Pool 13:  (17991, 27)
Now adding a year column
(17991, 28)
Adding a timecode column
(17991, 29)
Now filtering by backwater lakes
(8097, 29) [1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2004 2005 2006 2007
 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019]
Filtering out bad data
(813, 29) [1993 1994 1995 1996 1998 2000 2001 2002 2004 2005 2006 2007 2008 2009
 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019]
Dropping all blank columns
(813, 17)
Filtering out points with blank entries in at least one of the columns
(258, 17) [1993 1994 1995 1996]


In [63]:
data.to_csv("Cleaned_data_12-29-20.csv")

In [59]:
data.isna().sum()

FLDNUM       0
DATE         0
LOCATCD      0
WDP          0
SECCHI       0
STRATUM      0
TEMP         0
DO           0
TURB         0
COND         0
VEL          0
TP           0
TN           0
SS           0
CHLcal       0
YEAR         0
TIME CODE    0
dtype: int64

In [64]:
data.head()

,FLDNUM,DATE,LOCATCD,WDP,SECCHI,STRATUM,TEMP,DO,TURB,COND,VEL,TP,TN,SS,CHLcal,YEAR,TIME CODE
39655,3,08/04/1993,9332060,1.5,30.0,3,23.6,7.2,42.0,445.0,0.46,0.329,2.529,64.4,12.26380,1993,2
39661,3,08/04/1993,9332061,1.5,30.0,3,23.6,7.3,41.0,445.0,0.61,0.364,2.321,77.6,13.87240,1993,2
39666,3,08/04/1993,9332062,0.5,32.0,3,23.8,7.2,39.0,456.0,0.12,0.335,2.607,68.5,15.64186,1993,2
39687,3,08/04/1993,9332064,1.5,34.0,3,23.6,6.9,41.0,435.0,0.39,0.304,2.546,69.4,14.83756,1993,2
39690,3,08/04/1993,9332059,1.5,34.0,3,23.5,6.9,35.0,435.0,0.69,0.317,2.217,84.3,14.75713,1993,2
